In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [2]:
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 9.862 sec


In [3]:
# SQLAlchemy connection
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn = engine.connect()

In [7]:
CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""

In [9]:
dbcnxn = pyodbc.connect(CONNAS400)
cursor = dbcnxn.cursor()
dbase = []

In [10]:
strsql = """SELECT PROD.FPSPRMAST1.SPH_PART,
                       STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC1),
                       STRIP(PROD.FPSPRMAST1.SPH_DESC2),
                       STRIP(PROD.FPSPRMAST1.SPH_MFG),
                       STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
                       STRIP(PROD.FPSPRMAST2.SPD_CABINT),
                       STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
                       PROD.FPSPRMAST2.SPD_QOHCUR,
                       PROD.FPSPRMAST1.SPH_CURSTD,
                       STRIP(PROD.FPSPRMAST2.SPD_REODTE),
                       STRIP(PROD.FPSPRMAST2.SPD_USECC),
                       STRIP(PROD.FPSPRMAST2.SPD_PURCC),
                       STRIP(PROD.FPSPRMAST2.SPD_QREORD)
                FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
                WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))"""

In [4]:
sql = """
            SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
                "Screw Part Number" AS Screw, "Band Feed from Band data",
                "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
                "CameraInspectionRequired", "ScrDrvChk"
            FROM tbl8Tridon 
"""

In [11]:
cursor.execute(strsql)
result_spares = cursor.fetchall()

In [24]:
data_type_dict = {'StandardCost' : float, 'OnHand' : int, 'PartNum' : str}
df = pd.DataFrame.from_records(result_spares)
df.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand', 'StandardCost','ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
df = df.dropna()
df = df[1:]
df = df.astype(data_type_dict)
df = df.convert_dtypes()
df.dtypes
df.head(10)

,PartNum,EngPartNum,Desc1,Desc2,Mfg,MfgPn,Cabinet,Drawer,OnHand,StandardCost,ReOrderDate,DeptUse,DeptPurch,ReOrderPt
1,1107,091001107,PLATE SIDE WEAR,,,,',',0,95.0,000000,0750,0750,0
2,1114,091001114,PLATE SIDE WEAR FRONT,,,,',',0,90.0,000000,0750,0750,0
3,1115,091001115,PLATE WEAR SIDE,,,,C7,D1,4,90.0,000000,0750,0750,1
4,2001,4-40 H3,SP PT GUN TAP PLUG STYLE,MSC # 64932445,,,---,---,0,0.01,000000,0750,0750,0
5,2002,5-40 H3,SP PT GUN TAP PLUG STYLE,MSC PART # 64931538,,,---,---,0,6.56,000000,0750,0750,0
6,2003,6-32 H3,SP PT GUN TAP PLUG STYLE,MSC PART # 64932627,,,---,---,0,4.39,000000,0750,0750,0
7,2004,8-32 H3,SP PT GUN TAP PLUG STYLE,MSC PART # 64932726,,,---,---,0,5.0,000000,0750,0750,0
8,2005,10-24 H3,SP PT GUN TAP PLUG STYLE,MSC PART # 64932817,,,---,---,0,4.92,000000,0750,0750,0
9,2006,10-32 H3,SP PT GUN TAP PLUG STYLE,MSC PART # 64932833,,,---,---,0,4.82,000000,0750,0750,0
10,2007,1/4-20 H3,SP PT GUN TAP PLUG STYLE,MSC # 64938004,,,---,---,0,4.62,000000,0750,0750,0


In [5]:
start = timer()
cursor.execute(sql)
result = cursor.fetchall()

In [6]:
data_type_dict = {'Feed' : float}
df = pd.DataFrame.from_records(result)
df.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk']
df = df[df.Feed != 'N/A']
df = df.dropna()
df = df[1:]
df.fillna({'DiaMax' : 0.0}, inplace=True)
df.fillna({'DiaMin' : 0.0}, inplace=True)
df = df.astype(data_type_dict)
df = df.convert_dtypes()
df.head(20)
# Display Duplicated Part Numbers
# ids = df['PartNumber']
# df[ids.isin(ids[ids.duplicated()])].sort_values('PartNumber')


,PartNumber,Band,Housing,Screw,Feed,DiaMax,DiaMin,HexSz,BandThickness,BandWidth,CamInspect,ScrDrvChk
3,M674047102,1083047,2367012,1278001,12.165,0.0,0.0,"5/16""",0.025,0.5,NO,NO
4,M679064102,2019064,2367012,1278001MC,14.5,0.0,0.0,"5/16""",0.025,0.5,NO,NO
5,M674088102,1083088,2367012,1278001,19.988,0.0,0.0,"5/16""",0.025,0.5,NO,NO
6,M725104106,2009104,2367025,1278001,23.028,0.0,0.0,"5/16""",0.025,0.5,NO,NO
7,M674038102,1083038,2367012,1278001,9.98,0.0,0.0,"5/16""",0.025,0.5,NO,NO
8,M487056102,2001056,2057003,2215002,13.5,0.0,0.0,6 mm,0.025,0.4,YES,NO
9,M550056102,1208056,1209005,2375003VB,13.375,0.0,0.0,"5/16""",0.022,0.5,NO,NO
10,M550088102,1208088,1209005,2375003VB,19.78,0.0,0.0,"5/16""",0.022,0.5,NO,NO
11,M550188106,1208188,1209005,2375003VB,39.6,0.0,0.0,"5/16""",0.022,0.5,NO,NO
12,M604104106,2366104,2350012,1213002,22.96,0.0,0.0,"5/16""",0.022,0.5,NO,NO


In [25]:
df.to_sql('tblSpares', conn, schema='eng', if_exists='replace', index=False)

27

In [44]:
df.to_sql('parts_clamps', conn, schema='production', if_exists='replace', index=False)

169